rgb 256

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define dataset paths
train_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train"
val_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/valid"

# Define label mapping
label_mapping = {"FRESH": 0, "SPOILED": 1}

# Function to extract label from filename
def get_label_from_filename(filename):
    if "FRESH" in filename.upper():
        return label_mapping["FRESH"]
    elif "SPOILED" in filename.upper():
        return label_mapping["SPOILED"]
    else:
        return None  # Ignore unexpected filenames

# Function to load images and labels
def load_images(folder, img_size=(256, 256)):
    images, labels = [], []

    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)

        label = get_label_from_filename(file)
        if label is not None:
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
                img = cv2.resize(img, img_size)  # Resize image
                img = img / 255.0  # Normalize

                images.append(img)
                labels.append(label)

    return np.array(images), np.array(labels)

# Load train and validation data
X_train, y_train = load_images(train_dir, img_size=(256, 256))
X_val, y_val = load_images(val_dir, img_size=(256, 256))

print(f"Loaded {len(X_train)} training images and {len(X_val)} validation images")


Loaded 1815 training images and 451 validation images


In [ ]:
# Define CNN Model with 3 Convolutional Layers
def build_cnn(input_shape):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # Binary Classification (Fresh/Spoiled)
    ])
    return model

# Create and compile the model
model = build_cnn((256, 256, 3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
print("Training the CNN...")
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate model
test_loss, test_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {test_acc * 100:.2f}%")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training the CNN...
Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 312s 6s/step - accuracy: 0.6991 - loss: 0.9792 - val_accuracy: 0.7472 - val_loss: 0.5238
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 322s 6s/step - accuracy: 0.8303 - loss: 0.4170 - val_accuracy: 0.7517 - val_loss: 0.4386
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 319s 6s/step - accuracy: 0.8464 - loss: 0.3584 - val_accuracy: 0.8381 - val_loss: 0.3809
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 308s 6s/step - accuracy: 0.8985 - loss: 0.2500 - val_accuracy: 0.8736 - val_loss: 0.2766
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 319s 6s/step - accuracy: 0.9476 - loss: 0.1398 - val_accuracy: 0.9424 - val_loss: 0.1420
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 306s 6s/step - accuracy: 0.9719 - loss: 0.0740 - val_accuracy: 0.9446 - val_loss: 0.1605
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 306s 6s/step - accuracy: 0.9771 - loss: 0.0696 - val_accuracy: 0.9712 - val_loss: 0.0890
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 324s 6s/step - accuracy: 0.9882 - loss: 0.0363 - val_

In [ ]:
import requests
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from io import BytesIO

# Function to load and preprocess image from a URL or local path
def preprocess_image(path, target_size=(256, 256)):
    """
    Loads and preprocesses an image from a URL or local file path.

    Args:
        path (str): The URL or local file path of the image.
        target_size (tuple): The desired size of the image (width, height).

    Returns:
        numpy.ndarray: The preprocessed image as a NumPy array.
    """
    try:
        # Attempt to open the path as a URL
        response = requests.get(path)
        img = Image.open(BytesIO(response.content)).convert("RGB")
    except (requests.exceptions.MissingSchema, requests.exceptions.InvalidSchema):
        # If it's not a valid URL, assume it's a local file path
        img = Image.open(path).convert("RGB")

    img = img.resize(target_size)  # Resize image
    img_array = np.array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to make a prediction
def predict_image(path, model):
    img_array = preprocess_image(path)
    prediction = model.predict(img_array)[0][0]  # Get single prediction
    if prediction > 0.5:
        print("Prediction: ❌ Spoiled Meat")
    else:
        print("Prediction: ✅ Fresh Meat")

# Example usage
image_url = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train/FRESH-1-_JPG.rf.65663691924ca0aede3884b863267c98.jpg"  # Replace with actual image URL or local file path
predict_image(image_url, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 497ms/step
Prediction: ✅ Fresh Meat


greyscale 256

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define dataset paths
train_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train"
val_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/valid"

# Define label mapping
label_mapping = {"FRESH": 0, "SPOILED": 1}

# Function to extract label from filename
def get_label_from_filename(filename):
    if "FRESH" in filename.upper():
        return label_mapping["FRESH"]
    elif "SPOILED" in filename.upper():
        return label_mapping["SPOILED"]
    else:
        return None  # Ignore unexpected filenames

# Function to load images and labels (convert to grayscale)
def load_images(folder, img_size=(256, 256)):
    images, labels = [], []

    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)

        label = get_label_from_filename(file)
        if label is not None:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
            if img is not None:
                img = cv2.resize(img, img_size)  # Resize image
                img = img / 255.0  # Normalize

                images.append(img)
                labels.append(label)

    return np.array(images).reshape(-1, img_size[0], img_size[1], 1), np.array(labels)  # Add channel dimension

# Load train and validation data
X_train, y_train = load_images(train_dir, img_size=(256, 256))
X_val, y_val = load_images(val_dir, img_size=(256, 256))

print(f"Loaded {len(X_train)} training images and {len(X_val)} validation images")


In [ ]:
# Define CNN Model with 3 Convolutional Layers
def build_cnn(input_shape):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # Binary Classification (Fresh/Spoiled)
    ])
    return model

# Create and compile the model
model = build_cnn((256, 256, 3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
print("Training the CNN...")
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate model
test_loss, test_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {test_acc * 100:.2f}%")


In [ ]:
import requests
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from io import BytesIO

# Function to load and preprocess image from a URL or local path (grayscale)
def preprocess_image(path, target_size=(256, 256)):
    """
    Loads and preprocesses an image from a URL or local file path in grayscale.

    Args:
        path (str): The URL or local file path of the image.
        target_size (tuple): The desired size of the image (width, height).

    Returns:
        numpy.ndarray: The preprocessed image as a NumPy array.
    """
    try:
        # Attempt to open the path as a URL
        response = requests.get(path)
        img = Image.open(BytesIO(response.content)).convert("L")  # Convert to grayscale
    except (requests.exceptions.MissingSchema, requests.exceptions.InvalidSchema):
        # If it's not a valid URL, assume it's a local file path
        img = Image.open(path).convert("L")  # Convert to grayscale

    img = img.resize(target_size)  # Resize image
    img_array = np.array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=(0, -1))  # Add batch and channel dimensions (for CNN input)

    return img_array

# Function to make a prediction
def predict_image(path, model):
    img_array = preprocess_image(path)
    prediction = model.predict(img_array)[0][0]  # Get single prediction
    if prediction > 0.5:
        print("Prediction: ❌ Spoiled Meat")
    else:
        print("Prediction: ✅ Fresh Meat")

# Example usage
image_path = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train/FRESH-1-_JPG.rf.65663691924ca0aede3884b863267c98.jpg"  # Replace with actual image URL or local file path
predict_image(image_path, model)


rgb 200

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from io import BytesIO
import requests

# Define dataset paths
train_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train"
val_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/valid"

# Define label mapping
label_mapping = {"FRESH": 0, "SPOILED": 1}

# Function to extract label from filename
def get_label_from_filename(filename):
    if "FRESH" in filename.upper():
        return label_mapping["FRESH"]
    elif "SPOILED" in filename.upper():
        return label_mapping["SPOILED"]
    else:
        return None  # Ignore unexpected filenames

# Function to load images and labels
def load_images(folder, img_size=(200, 200)):  # Changed image size to 200x200
    images, labels = [], []

    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)

        label = get_label_from_filename(file)
        if label is not None:
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
                img = cv2.resize(img, img_size)  # Resize image to 200x200
                img = img / 255.0  # Normalize

                images.append(img)
                labels.append(label)

    return np.array(images), np.array(labels)

# Load train and validation data
X_train, y_train = load_images(train_dir, img_size=(200, 200))
X_val, y_val = load_images(val_dir, img_size=(200, 200))

print(f"Loaded {len(X_train)} training images and {len(X_val)} validation images")

# Define CNN Model with 3 Convolutional Layers
def build_cnn(input_shape):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # Binary Classification (Fresh/Spoiled)
    ])
    return model

# Create and compile the model
model = build_cnn((200, 200, 3))  # Changed input shape to 200x200
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
print("Training the CNN...")
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate model
test_loss, test_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {test_acc * 100:.2f}%")

# Function to load and preprocess image from a URL or local path
def preprocess_image(path, target_size=(200, 200)):  # Changed target size to 200x200
    """
    Loads and preprocesses an image from a URL or local file path.

    Args:
        path (str): The URL or local file path of the image.
        target_size (tuple): The desired size of the image (width, height).

    Returns:
        numpy.ndarray: The preprocessed image as a NumPy array.
    """
    try:
        # Attempt to open the path as a URL
        response = requests.get(path)
        img = Image.open(BytesIO(response.content)).convert("RGB")
    except (requests.exceptions.MissingSchema, requests.exceptions.InvalidSchema):
        # If it's not a valid URL, assume it's a local file path
        img = Image.open(path).convert("RGB")

    img = img.resize(target_size)  # Resize image to 200x200
    img_array = np.array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to make a prediction
def predict_image(path, model):
    img_array = preprocess_image(path)
    prediction = model.predict(img_array)[0][0]  # Get single prediction
    if prediction > 0.5:
        print("Prediction: ❌ Spoiled Meat")
    else:
        print("Prediction: ✅ Fresh Meat")

# Example usage
image_path = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train/FRESH-1-_JPG.rf.65663691924ca0aede3884b863267c98.jpg"  # Replace with actual image path
predict_image(image_path, model)


greyscale 200

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from io import BytesIO
import requests

# Define dataset paths
train_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train"
val_dir = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/valid"

# Define label mapping
label_mapping = {"FRESH": 0, "SPOILED": 1}

# Function to extract label from filename
def get_label_from_filename(filename):
    if "FRESH" in filename.upper():
        return label_mapping["FRESH"]
    elif "SPOILED" in filename.upper():
        return label_mapping["SPOILED"]
    else:
        return None  # Ignore unexpected filenames

# Function to load images and labels (convert to grayscale)
def load_images(folder, img_size=(200, 200)):
    images, labels = [], []

    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)

        label = get_label_from_filename(file)
        if label is not None:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
            if img is not None:
                img = cv2.resize(img, img_size)  # Resize image to 200x200
                img = img / 255.0  # Normalize

                images.append(img)
                labels.append(label)

    return np.array(images).reshape(-1, 200, 200, 1), np.array(labels)  # Reshape for CNN

# Load train and validation data
X_train, y_train = load_images(train_dir, img_size=(200, 200))
X_val, y_val = load_images(val_dir, img_size=(200, 200))

print(f"Loaded {len(X_train)} training images and {len(X_val)} validation images")

# Define CNN Model with 3 Convolutional Layers
def build_cnn(input_shape):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # Binary Classification (Fresh/Spoiled)
    ])
    return model

# Create and compile the model
model = build_cnn((200, 200, 1))  # Changed input shape to (200, 200, 1) for grayscale
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
print("Training the CNN...")
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate model
test_loss, test_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {test_acc * 100:.2f}%")

# Function to load and preprocess image from a URL or local path (convert to grayscale)
def preprocess_image(path, target_size=(200, 200)):
    """
    Loads and preprocesses an image from a URL or local file path.

    Args:
        path (str): The URL or local file path of the image.
        target_size (tuple): The desired size of the image (width, height).

    Returns:
        numpy.ndarray: The preprocessed image as a NumPy array.
    """
    try:
        # Attempt to open the path as a URL
        response = requests.get(path)
        img = Image.open(BytesIO(response.content)).convert("L")  # Convert to grayscale
    except (requests.exceptions.MissingSchema, requests.exceptions.InvalidSchema):
        # If it's not a valid URL, assume it's a local file path
        img = Image.open(path).convert("L")  # Convert to grayscale

    img = img.resize(target_size)  # Resize image to 200x200
    img_array = np.array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=(0, -1))  # Add batch and channel dimensions
    return img_array

# Function to make a prediction
def predict_image(path, model):
    img_array = preprocess_image(path)
    prediction = model.predict(img_array)[0][0]  # Get single prediction
    if prediction > 0.5:
        print("Prediction: ❌ Spoiled Meat")
    else:
        print("Prediction: ✅ Fresh Meat")

# Example usage
image_path = "/content/drive/MyDrive/Meat Freshness.v1-new-dataset.multiclass/train/FRESH-1-_JPG.rf.65663691924ca0aede3884b863267c98.jpg"
predict_image(image_path, model)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Assuming you have:
# y_test → true labels
# y_pred → predicted labels

print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='macro'))  # use 'binary' for 2 classes
print("Recall   :", recall_score(y_test, y_pred, average='macro'))
print("F1 Score :", f1_score(y_test, y_pred, average='macro'))

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats.weightstats import ztest

np.random.seed(1)

# Simulate "freshness score" feature for 2 classes
fresh_scores = np.random.normal(loc=85, scale=5, size=50)   # Higher score = more fresh
spoiled_scores = np.random.normal(loc=60, scale=7, size=50) # Lower score = more spoiled

# Combine into DataFrame
df = pd.DataFrame({
    'FreshnessScore': np.concatenate((fresh_scores, spoiled_scores)),
    'Label': ['Fresh']*50 + ['Spoiled']*50
})


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Create confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Visualize
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


NameError: name 'y_test' is not defined

In [ ]:
z_stat, p_val = ztest(spoiled_scores, value=65)
print(f"Z-Test: z = {z_stat:.2f}, p = {p_val:.4f}")


Z-Test: z = -5.08, p = 0.0000


In [ ]:
t_stat, p_val = stats.ttest_ind(fresh_scores, spoiled_scores)
print(f"T-Test: t = {t_stat:.2f}, p = {p_val:.4f}")


T-Test: t = 22.82, p = 0.0000


In [ ]:
f_stat = np.var(fresh_scores, ddof=1) / np.var(spoiled_scores, ddof=1)
df1, df2 = len(fresh_scores) - 1, len(spoiled_scores) - 1
p_val = 1 - stats.f.cdf(f_stat, df1, df2)
print(f"F-Test: F = {f_stat:.2f}, p = {p_val:.4f}")


F-Test: F = 0.78, p = 0.8018


In [ ]:
# Simulate 3 groups
group1 = np.random.normal(85, 5, 30)  # Fresh
group2 = np.random.normal(70, 5, 30)  # Slightly Spoiled
group3 = np.random.normal(55, 6, 30)  # Heavily Spoiled

f_stat, p_val = stats.f_oneway(group1, group2, group3)
print(f"ANOVA Test: F = {f_stat:.2f}, p = {p_val:.4f}")


ANOVA Test: F = 255.88, p = 0.0000
